In [1]:

using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials

In [5]:

# import conventions
include("conventions.jl")
  using .conventions: big_endian, qubit_begin

  # import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: Qgate_T2D

include("quantum_circuit.jl")
using ..quantum_circuit: qc_initialize, init_register, show_statevector, apply_op
#using ..quantum_circuit: qc_initialize, init_register, print_initstate

Load quantum gates constructor
Load Tensor module: QTensor.jl


Load quantum gates constructor
Load quantum gates constructor
Load quantum gates constructor


### quantum sensor test 

In [7]:

function qc_initialize_test(n::Int64, 
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol,
    q_order::String="big-endian")
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end
        
   else
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 
end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

#    return 1

end # end qc_initialize  


qc_initialize_test (generic function with 4 methods)

In [2]:
qubit_control = 0
qubit_target = 1
nqubits = 3
err_tol = 1e-8
qc = qc_initialize(nqubits)


Main.quantum_circuit.qc_initstruct(3, "big-endian", 2, 8, [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0 0.0 0.0; 0.0 0.0 1.0; … ; 1.0 1.0 0.0; 1.0 1.0 1.0])

#### testing the rotational gate
$R_Z(\phi)$.

syntax for the gate is: `Rz_gate = Qgate.Rz(phi)`

where `phi` is the angle of rotation.

In [6]:
Rz_gate = Qgate.Rz(2.0)


2×2 Matrix{ComplexF64}:
 1.0+0.0im        0.0-0.0im
 0.0+0.0im  -0.416147+0.909297im

In [8]:
# a gate from the Qgate library implemented from the paper 
# arXiv:2209.08187v1 [quant-ph] 16 Sep 2022
Rz_gate = Rz_gate1(2.0)

2×2 Matrix{ComplexF64}:
 0.540302-0.841471im       0.0+0.0im
      0.0+0.0im       0.540302+0.841471im

### ===============================

Now, we start with the implementation of the quantum circuit simulating the quantum sensor.

In [9]:
qubit_control = 0
qubit_target = 1
# Start with one qubit only and one classical bit to recieve the answer
nqubits = 1
err_tol = 1e-8
qc = qc_initialize(nqubits)

Main.quantum_circuit.qc_initstruct(1, "big-endian", 2, 2, [1.0, 0.0], [0, 1])

In [10]:
# check the state vector of the quantum register
qc.state_vector

2-element Vector{Float64}:
 1.0
 0.0

In [11]:
# check the quantum states, the basis of the Hilbert space of the qubits
qc.q_states

2-element Vector{Int64}:
 0
 1

In [13]:
# print the state vector with the basis of the quantum register
show_statevector(qc)

The initial state of the quantum register is: 
The initial state of the quantum register with the 
    quantum states in the computational basis is: 
1.0 * | [0]>
0.0 * | [1]>


In [14]:
# Get the Hadamard gate
Hgate = Qgate.H

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107

In [15]:
# Apply the Hadamard gate to the quantum register
apply_op(qc, Hgate)

Main.quantum_circuit.qc_initstruct(1, "big-endian", 2, 2, [0.7071067811865475, 0.7071067811865475], [0, 1])

In [18]:
# Show the state vector of the quantum register
show_statevector(qc)

The initial state of the quantum register is: 
The initial state of the quantum register with the 
    quantum states in the computational basis is: 
0.7071067811865475 * | [0]>
0.7071067811865475 * | [1]>


In [ ]:
# test the unitary condition of the rotational gate


In [17]:
# Apply the Rz gate as defined in the paper arXiv:2209.08187v1 [quant-ph] 16 Sep 2022
apply_op(qc, Rz_gate)

ErrorException: The quantum gate is not unitary

## refs

https://quantumcomputing.stackexchange.com/questions/14066/how-do-i-apply-the-hadamard-gate-to-one-qubit-in-a-two-qubit-pure-state
